# Demo guide for package minplascalc

Welcome to minplascalc, a Python 3 package to assist with calculations of equilibrium compositions and thermophysical properties of thermal plasmas of the sort typically encountered in metallurgical processing applications.

## 1. Target audience

Plasma technology researchers and professionals with a basic knowledge of the Python programming language.

## 2. Description

Ionised gases or <i>plasmas</i> are used in many industrial applications such as arc welding, plasma spraying, and electric furnace metallurgy. Engineering plasmas typically operate at atmospheric pressures and temperatures of the order of 10<sup>4</sup> K. Thermal plasmas of the sort considered here are assumed to be in local thermodynamic equilibirum (LTE), meaning that a single unique temperature can be used to describe them (as opposed to two-temperature plasmas in which the electron temperature is distinct from the heavy-particle temperature). A major advantage of the LTE approximation is that all thermophysical properties of an equilibrium mixture of an arbitrary number of plasma species can be expressed as (complicated) functions of temperature and pressure only.

Knowledge of these thermophysical properties is of great value to engineers working in plasma technology. Such information is useful for simple design calculations, and is necessary input data for computational fluid dynamics and magnetohydrodynamics models of plasma systems. The calculation of composition and thence the thermophysical properties of a thermal plasma given some fundamental information about the species present is a well-understood but mathematically and numerically complex process - it is prone to error if performed manually, hence the need for this tool.

Things you <b>can</b> calculate with minplascalc:
* Statistical mechanics partition functions for individual species using information about the energy levels and excited states
* Equilibrium plasma composition in terms of number densities of a specified mixture of species, using the principle of Gibbs free energy minimisation at a specified temperature and pressure
* Density $\rho$, (relative) enthalpy $H$, and heat capacity $C_P$ at equilibrium conditions

Things you <b>can't</b> calculate with minplascalc:
* Compositions or thermophysical properties of two-temperature plasmas
* $C_P$ for arbitrary non-LTE compositions
* Plasmas of complex molecules or molecular ions consisting of more than two atoms
* Plasmas involving negatively-charged ions
* Transport or thermal radiation properties (see future versions though)

The package also includes some basic tools to format and import data for individual species obtained from the NIST Atomic Spectra and Chemistry WebBook databases, and store it for use by future simulations.

## 3. Worked examples

Some examples of how to use various parts of the minplascalc package follows.

*General utilities and species calculations*
* <a href="#3.1a">Add a new monatomic species data entry</a>
* <a href="#3.1b">Add a new diatomic species data entry</a>
* <a href="#3.2">Calculate an internal partition function</a>

*Example 1 - oxygen plasma*
* <a href="#3.3">Calculate an equilibrium composition</a>
* <a href="#3.4">Calcluate density and heat capacity at equilibrium</a>

*Example 2 - silicon & carbon monoxide plasma*
* <a href="#3.5">Calculate an equilibrium composition</a>
* <a href="#3.6">Calcluate density and heat capacity at equilibrium</a>

### 3.1 Adding a new species entry to your local data store

If you would like to do calculations using a plasma species that you haven't used before, you will need to generate a minplascalc data entry for it first. You only need to do this <i>once</i> - minplascalc will store the species data in a plain-text file formatted using JSON syntax, and it will then be available for use in any of your future calculations.

<a id="3.1a"></a>
#### 3.1a Add a monatomic species

minplascalc has built-in readers for handling data obtained from the energy levels section of NIST Atomic Spectra Database, which can be found at http://physics.nist.gov/PhysRefData/ASD/levels_form.html. The landing page looks like this:

<img src="demoimg/asd_landingpage.png" alt="NIST ASD landing page" style="width: 800px;"/>

You must then specify the atom or ion that you want to retrieve energy level information for. Let's get the data for the singly-charged oxygen cation species O<sup>+</sup>, which is "O II" in spectrographic terminology. Enter the identifier in the Spectrum field on the form:

<img src="demoimg/asd_spectrumidentification.png" alt="Spectrum identification" style="width: 400px;"/>

Make sure you set Level Units to "cm-1", Format Output to "ASCII (text)", and uncheck everything in the output section except Level and J, like this:

<img src="demoimg/asd_retrievalsettings.png" alt="Spectrum identification" style="width: 400px;"/>

Click the Retrieve Data button. You should see a page with the energy levels listed in a plain-text table. Click and drag to select everything from the first energy level line down to the last one before the first ionisation energy entry (these have no J value and are separated in boxes of ASCII dashes). While you're there, make a note of the ionisation energy value.

<table style="border:none!important">
<td><img src="demoimg/asd_leveldata1.png" alt="Spectrum identification" style="height: 400px;"/>
<td><img src="demoimg/asd_leveldata2.png" alt="Spectrum identification" style="height: 400px;"/>
<td><img src="demoimg/asd_leveldata3.png" alt="Spectrum identification" style="height: 400px;"/>
</table>

Copy and paste the selected content into a temporary text file. This has already been done for the oxygen cation, and the raw NIST data file is located at minplascalc/data/species_raw/nist_O+. Running the following code snippet will add a data entry for the O<sup>+</sup> ion to minplascalc:

In [3]:
import minplascalc as mpc

mpc.buildMonatomicSpeciesJSON(
    name="O+",
    stoichiometry={"O": 1},
    molarMass=0.0159994,
    chargeNumber=1,
    ionisationEnergy=283270.9,
    nistDataFile="minplascalc/data/species_raw/nist_O+")


What's happening here? First we import the minplascalc package, then we call a function to build and save the species data entry for us. 

The function takes as arguments the species name (which is also used as the data entry's associated file name), a small dictionary describing the elemental stoichiometry of the species (in this case a single oxygen atom), the molar mass in kg/mol, the charge on the species in units of the fundamental charge (in this case 1 because O<sup>+</sup> is singly charged), the ionisation energy of the species in cm<sup>-1</sup>, and the path to the energy level text file we just generated from the NIST site. Internally to minplascalc the function reads the energy levels from the text file, assembles the species information into a Python dictionary format, and writes it out to a JSON file called "O+.json" on the path you're running this notebook from.

After this process it will be possible to create an O<sup>+</sup> species object in any minplascalc calculation by importing it using either the explicit path to the JSON file, or (preferably) just the name of the species provided the JSON file is stored in any of the standard minplascalc data paths - see later demos for examples.

<a id="3.1b"></a>
#### 3.1b Diatomic species

The procedure for generating a minplascalc data entry for a new diatomic species is more manual since the data required must be retrieved from various locations. The majority of data for a wide range of species is available from the NIST Chemistry WebBook, http://webbook.nist.gov/chemistry/form-ser/. Enter the formula for the diatomic species you're interested in in the search box at the top of the page, and click the Search button. If you're looking for data for charged diatomic ions, make sure the "Exclude ions from the search" box is unchecked. Let's build a data entry for Silicon Monoxide, SiO:

<table>
<td><img src="demoimg/webbook_landingpage.png" alt="NIST Chemistry WebBook" style="height: 300px;"/>
<td><img src="demoimg/webbook_sio.png" alt="SiO WebBook entry" style="height: 300px;"/>
</table>

The first pieces of information needed are the ionisation energy (to turn SiO into SiO<sup>+</sup>) and the bond dissociation energy (to turn SiO into monatomic Si and O), both in units of cm<sup>-1</sup>. The ionisation energy can be found on the "Gas phase ion energetics" link on the Chemistry WebBook species page (usually in units of eV, so remember to convert):

<img src="demoimg/webbook_ionenergetics.png" alt="SiO ionisation energy" style="width: 400px;"/>

The dissociation energy can be found either on the NIST Computational Chemistry Comparison and Benchmark Database (http://cccbdb.nist.gov/introx.asp), or alternatively from chemistry textbooks or other sources. In the case of SiO, the value is 66707.6 cm<sup>-1</sup> (approximately 800 kJ/mol).

We then need some information about the electronic ground state of the molecule, and its vibrational and rotational parameters. To get this, go back to the main page for SiO in the Chemistry WebBook and click on "Constants of diatomic molecules". This gives the parameters of various energetic states of the molecule - we want the ground state with energy $T_e$ = 0, so scroll all the way down to the entry at the very bottom:

<table>
<td><img src="demoimg/webbook_constants1.png" alt="Molecule constants" style="height: 250px;"/>
<td><img src="demoimg/webbook_constants2.png" alt="Ground state level entry" style="height: 250px;"/>
</table>

The electronic degeneracy of the ground state, $g_0$, is given by the superscriped number in the expression in the "State" column - in the case of SiO, it's 1. The vibrational constant $\omega_e$ and rotational constant $B_e$ are read off the appropriate columns for the ground state entry. The symmetry constant $\sigma_s$ takes the value of 1 for heteronuclear molecules like SiO, and 2 for homonuclear molecules like O<sub>2</sub>. Now we have all the information to build a minplascalc data entry for the SiO species, which can be done by running the following code snippet:

In [2]:
import minplascalc as mpc

mpc.buildDiatomicSpeciesJSON(
    name="SiO",
    stoichiometry={"Si": 1, "O": 1},
    molarMass=0.0440849,
    chargeNumber=0,
    ionisationEnergy=92673.11,
    dissociationEnergy=66707.6,
    sigmaS=1,
    g0=1,
    we=1241.55,
    Be=0.7267512)


What's happening here? As before we import the minplascalc package, then we call a function to build and save the species data entry for us. 

The function takes as arguments the species name (which is also used as the data entry's associated file name), a small dictionary describing the elemental stoichiometry of the species (in this case one oxygen and one silicon), the molar mass in kg/mol, the charge on the species in units of the fundamental charge (in this case 0 because SiO is a neutral species), the ionisation and dissociation energies of the species in cm<sup>-1</sup>, and the electronic, vibrational, and rotational energy level parameters looked up from the Chemistry WebBook. Internally to minplascalc the function assembles the species information into a Python dictionary format, and writes it out to a JSON file called "SiO.json" on the path you're running this notebook from.

After this process it will be possible to create an SiO species object in any minplascalc calculation by importing it using either the explicit path to the JSON file, or (preferably) just the name of the species provided the JSON file is stored in any of the standard minplascalc data paths - see later demos for examples.

<a id="3.2"></a>
### 3.2 Calculating partition functions for a single species

Statistical mechanics partition functions for each of the species present are a core component of thermal plasma calculations. In general users of minplascalc should not need to access these functions explicitly as they are incorporated directly into the composition and thermophysical property calculators, but they are exposed in the API in case the need to do so ever arises.

#### Partition function descriptions in minplascalc

Recall that the partition function for a particular species is a description of the statistical properties of a collection of atoms or molecules of that species at thermodynamic equilibrium. Partition functions are generally presented as the sum of weighted state probabilities across the possible energy states of the system. In general at moderate plasma temperatures up to a few 10<sup>4</sup> K, a species' total partition function $Q_{tot}$ can be written as the product of several unique partition functions arising from different quantum mechanical phenomena (the weak-coupling approximation):

$Q_{tot} = Q_t Q_{int} = Q_t Q_e Q_v Q_r$

Here, $Q_t$ is the translational partition function due to the species' ability to move around in space, $Q_{int}$ is the internal partition function due to energy states internal to the particles of the species, $Q_e$ is the electronic partition function due to different possible arrangements of the electron structure of the species, $Q_v$ is the vibrational partition function due to the ability of the bonds in a polyatomic species to vibrate at different energy levels, and $Q_r$ is the rotational partition function due to a species' ability to rotate around its center of mass at different energy levels.

minplascalc distinguishes between three different types of species - monatomic (single atoms and ions), diatomic (bonded pairs of atoms), and free electrons. The formulae used for the various partition functions for each are shown in the table below.

<table style="width: 80%;">
<tr><td>Partition Function<td>Monatomic<td>Diatomic<td>Electron<tr>
<td>$Q_t$, m<sup>-3</sup>
<td>$${\left ( \frac{2 \pi m_s k_B T}{h^2}\right )}^{\frac{3}{2}}$$
<td>$${\left ( \frac{2 \pi m_s k_B T}{h^2}\right )}^{\frac{3}{2}}$$
<td>$${\left ( \frac{2 \pi m_e k_B T}{h^2}\right )}^{\frac{3}{2}}$$<tr>
<td>$Q_e$, dim'less
<td>$$\sum_j g_j \exp \left(-\frac{E_j}{k_B T}\right)$$
<td>$$g_0$$
<td>$$2$$<tr>
<td>$Q_v$, dim'less
<td>$$1$$
<td>$$\frac{1}{1-\exp\left( -\frac{\omega_e}{k_B T} \right)}$$
<td>$$1$$<tr>
<td>$Q_r$, dim'less
<td>$$1$$
<td>$$\frac{k_B T}{\sigma_s B_e}$$
<td>$$1$$</table>

Here $m_s$ and $m_e$ are the mass of one particle of the species, $k_B$ is the Boltzmann constant, $T$ is temperature, $h$ is the Planck constant, $g_j$ and $E_j$ are the degeneracy and energy of energy level j, and $\omega_e$ and $B_e$ are the vibrational and rotational constants respectively for a diatomic molecule.


<a id="3.3"></a>
### 3.3 Calculating the composition of an equilibrium plasma

[TODO]

<a id="3.4"></a>
### 3.4 Calculating the density and heat capacity of an equilibrium plasma

[TODO]